In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import numpy.ma as ma
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.model_selection import train_test_split

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
csgo_results = pd.read_csv("/kaggle/input/csgo-professional-matches/economy.csv")

#convert 1_winner to 1_wins and convert to bin from 1,2
#csgo_results["1_wins"] = csgo_results["1_winner"].apply(lambda x: 0 if x == 2.0 else 1)
#csgo_results = csgo_results.drop("1_winner", axis = 1)

#y is the target of the prediction
y = csgo_results["1_winner"]

#X is data used to determine y
X = csgo_results[["team_1","team_2","_map","t1_start","1_t1","1_t2"]]

In [ ]:
#Create a list of all teams and number them
set_of_teams = set(list(X['team_1']) + list(X['team_2']))
team_to_num = dict(zip(set_of_teams, range(len(set_of_teams))))

In [ ]:
#Replace team's name with their number
X['team_1'] = X['team_1'].replace(team_to_num)
X['team_2'] = X['team_2'].replace(team_to_num)

In [ ]:
#Convert from t1_start to t1_t and convert from t,ct to binary
X['t1_t'] = X['t1_start'].apply(lambda x: 1 if x == 't' else 0)
X = X.drop('t1_start', axis=1)

In [ ]:
#Replace map names with numbers
map_to_num = dict(zip(X['_map'].unique(), range(len(X['_map'].unique()))))
X['_map'] = X['_map'].replace(map_to_num)

In [ ]:
#Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [ ]:
test_accuracy = {}
for i in range(29):
    i+=2
    csgo_results[f"{i}_winner"]=np.nan_to_num(csgo_results[f"{i}_winner"])
    X[[f"{i}_winner"]] = csgo_results[[f"{i}_winner"]]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

    #Naive bayes algorithm
    from sklearn.naive_bayes import GaussianNB

    gnb = GaussianNB()
    gnb.fit(X_train, y_train)
    y_pred = gnb.predict(X_test)
    test_accuracy[i] = np.mean(y_pred == y_test)
    
    X = X.drop(f'{i}_winner', axis=1)

test_accuracy

In [ ]:
#Random forest classifier algorithm
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(max_depth=10)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
test_accuracy = np.mean(y_pred == y_test)
test_accuracy

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.preprocessing import StandardScaler
from datetime import *

csgo_players = pd.read_csv("/kaggle/input/csgo-professional-matches/players.csv")
scaler = StandardScaler()
csgo_players = csgo_players.dropna()
y = csgo_players['m1_rating']
X = csgo_players[['player_id','m1_kills','m1_assists','m1_deaths','m1_hs','m1_kast','m1_kddiff','m1_fkdiff']]
X = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

clf = LinearRegression()
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
test_accuracy = clf.score(X_test,y_test)
test_accuracy

In [ ]:
clf.coef_

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier

scaler = StandardScaler()


map_columns = ['t1_removed_1','t1_removed_2','t2_removed_1','t2_removed_2','t1_picked_1', 't2_picked_1', 'left_over']
csgo_maps = pd.read_csv("/kaggle/input/csgo-professional-matches/picks.csv")
csgo_maps = csgo_maps[csgo_maps['best_of']=='3']

#cleaning data
csgo_maps = csgo_maps[csgo_maps['t2_removed_2'] != '0.0']
csgo_maps = csgo_maps[csgo_maps['left_over'] != 'Luminosity']
csgo_maps = csgo_maps[csgo_maps['left_over'] != 'Alma']
csgo_maps = csgo_maps[csgo_maps['date']>='2019-07-15'].reset_index(drop=True)
csgo_maps = csgo_maps.drop(csgo_maps.index[755])
csgo_maps = csgo_maps[csgo_maps['date']>='2019-07-15'].reset_index(drop=True)

#changes all team names to unique number
team_list = list(set(np.concatenate((csgo_maps['team_1'].unique(), csgo_maps['team_2'].unique()), axis = 0)))
csgo_maps[['team_1', 'team_2']] = csgo_maps[['team_1', 'team_2']].apply(lambda j: j.replace(team_to_num))
team_to_num = dict(zip(team_list, range(len(team_list))))


csgo_results = pd.read_csv("/kaggle/input/csgo-professional-matches/economy.csv")
X = csgo_results[["team_1","team_2","_map","t1_start","1_t1","1_t2"]]
maps_to_num = dict(zip(X['_map'].unique(), range(len(X['_map'].unique()))))
#Replace map names with numbers
csgo_maps['map_order'] = csgo_maps[map_columns].apply(lambda x: x.replace(map_to_num)).apply(lambda y: y.values, axis=1)
csgo_maps[map_columns] = csgo_maps[map_columns].apply(lambda x: x.replace(map_to_num))



#takes list map_order and turns it to a string of the concatenated map numbers
csgo_maps['map_order'] = csgo_maps['map_order'].apply(lambda x: ''.join([str(i) for i in x]))
#tries to turn the strings back to a single int (unsuccessfully)
csgo_maps['map_order'] = csgo_maps['map_order'].astype(int)


x = csgo_maps[['team_1','team_2']]
y = csgo_maps[['map_order']]

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=42)

lr = LinearRegression()
rf = RandomForestRegressor()
knn = KNeighborsRegressor()
dt = DecisionTreeRegressor()
rfc = RandomForestClassifier(max_depth=10)
regressors = [lr, rf, knn, dt, rfc]
for regressor in regressors:
    regressor.fit(X_train, y_train)
    regressor.predict(X_test)
    score = regressor.score(X_test, y_test)
    print(score)
csgo_maps
# pred = LinearRegression()
# pred.fit(X_train, y_train)
# pred.predict(X_test)
# score = pred.score(X_test, y_test)


In [ ]:
new_matches = {'team_1': ['BIG', 'Vitality', 'Complexity', 'GODSENT', 'Astralis', 'FURIA'], 'team_2': ['FaZe', 'BIG', 'FaZe', 'Heroic', 'MIBR', 'G2']}

new_matches = pd.DataFrame(data=new_matches)
new_matches

In [ ]:
csgo_maps

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

map_columns = ['t1_removed_1','t1_removed_2','t2_removed_1','t2_removed_2','t1_picked_1', 't2_picked_1', 'left_over']
csgo_maps = pd.read_csv("/kaggle/input/csgo-professional-matches/picks.csv")
csgo_results = pd.read_csv("/kaggle/input/csgo-professional-matches/results.csv")
csgo_maps = csgo_maps[csgo_maps['best_of']=='3']


#cleaning the data
csgo_maps = csgo_maps[csgo_maps['t2_removed_2'] != '0.0']
csgo_maps = csgo_maps[csgo_maps['left_over'] != 'Luminosity']
csgo_maps = csgo_maps[csgo_maps['left_over'] != 'Alma']
csgo_maps = csgo_maps[csgo_maps['date']>='2019-07-15'].reset_index(drop=True)
csgo_maps = csgo_maps.drop(csgo_maps.index[755])
csgo_maps = csgo_maps[csgo_maps['date']>='2019-07-15'].reset_index(drop=True)
csgo_results = csgo_results[csgo_results['match_id'].isin(csgo_maps['match_id'])].groupby('match_id', group_keys=False).apply(lambda df: df.sample(1))


#changes all team names to unique number
team_list = list(set(np.concatenate((csgo_maps['team_1'].unique(), csgo_maps['team_2'].unique()), axis = 0)))
csgo_maps[['team_1', 'team_2']] = csgo_maps[['team_1', 'team_2']].apply(lambda j: j.replace(team_to_num))
team_to_num = dict(zip(team_list, range(len(team_list))))

csgo_econ = pd.read_csv("/kaggle/input/csgo-professional-matches/economy.csv")
X = csgo_econ[["team_1","team_2","_map","t1_start","1_t1","1_t2"]]
maps_to_num = dict(zip(X['_map'].unique(), range(len(X['_map'].unique()))))
#Replace map names with numbers
csgo_maps[map_columns] = csgo_maps[map_columns].apply(lambda x: x.replace(map_to_num))

X_maps = csgo_maps[['team_1','team_2','t1_removed_1','t1_removed_2','t2_removed_1','t2_removed_2','t1_picked_1', 't2_picked_1', 'left_over', 'match_id']]
X_results = csgo_results[['match_id', 'rank_1', 'rank_2', 'match_winner']]

csgo_df = X_maps.merge(X_results, on='match_id')
csgo_df